In [9]:
import os
import json
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()

True

In [2]:
llm1 = init_chat_model(model = os.getenv("GROQ_MODEL_NAME"), model_provider = "groq")
llm2 = init_chat_model(model = os.getenv("GEMINI_MODEL_NAME"), model_provider = "google_genai")

In [3]:
DATA_DIR = "data"
DELIVERIES_FILE = os.path.join(DATA_DIR, "deliveries.json")

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

deliveries = load_json(DELIVERIES_FILE)

prompt = "You are an operations planner. Rank deliveries with ids and reasons based on priority, location proximity and operator instructions.\n"
prompt += f"Operator instructions: \n\nDeliveries:\n"
for d in deliveries:
    prompt += f"- id:{d['id']}, priority:{d.get('priority','medium')}, lat:{d['lat']}, lon:{d['lon']}, package_size:{d.get('package_size','medium')}\n"
prompt += "\nReturn a JSON array of ids in preferred visit order."
print(prompt)

You are an operations planner. Rank deliveries with ids and reasons based on priority, location proximity and operator instructions.
Operator instructions: 

Deliveries:
- id:D001, priority:high, lat:28.6315, lon:77.2167, package_size:small
- id:D002, priority:medium, lat:28.5703, lon:77.324, package_size:medium
- id:D003, priority:low, lat:28.4936, lon:77.0896, package_size:large
- id:D004, priority:medium, lat:28.5682, lon:77.2378, package_size:small

Return a JSON array of ids in preferred visit order.


In [ ]:
resp1 = llm1.invoke([{"role":"user","content":prompt}])
resp2 = llm2.invoke([{"role":"user","content":prompt}])
text1 = str(resp1.content).strip()
text2 = str(resp2.content).strip()
print("Groq Response:", text1)
print("Gemini Response:", text2)


```json
["D001", "D004", "D002", "D003"]
```
As an operations planner, I will prioritize deliveries based on the given criteria: priority (high > medium > low) and then location proximity to create an efficient route. Since "Operator instructions" are empty, I will not factor in any additional specific operator preferences.

Here's the breakdown:

1.  **Group by Priority:**
    *   **High Priority:** D001 (28.6315, 77.2167)
    *   **Medium Priority:** D002 (28.5703, 77.324), D004 (28.5682, 77.2378)
    *   **Low Priority:** D003 (28.4936, 77.0896)

2.  **Determine Visit Order:**

    *   **Start with High Priority:**
        *   **D001** is the only high-priority delivery, so it comes first.

    *   **Next, consider Medium Priority (D002, D004) based on proximity to D001:**
        *   From D001 (28.6315, 77.2167):
            *   **D004** (28.5682, 77.2378) is geographically closer to D001 (small latitude and longitude difference) than D002.
            *   **D002** (28.5703, 77.324

In [19]:
print("Groq Response:", text1)
print("Gemini Response:", text2)

Groq Response: ```json
["D001", "D004", "D002", "D003"]
```
Gemini Response: As an operations planner, I will prioritize deliveries based on the given criteria: priority (high > medium > low) and then location proximity to create an efficient route. Since "Operator instructions" are empty, I will not factor in any additional specific operator preferences.

Here's the breakdown:

1.  **Group by Priority:**
    *   **High Priority:** D001 (28.6315, 77.2167)
    *   **Medium Priority:** D002 (28.5703, 77.324), D004 (28.5682, 77.2378)
    *   **Low Priority:** D003 (28.4936, 77.0896)

2.  **Determine Visit Order:**

    *   **Start with High Priority:**
        *   **D001** is the only high-priority delivery, so it comes first.

    *   **Next, consider Medium Priority (D002, D004) based on proximity to D001:**
        *   From D001 (28.6315, 77.2167):
            *   **D004** (28.5682, 77.2378) is geographically closer to D001 (small latitude and longitude difference) than D002.
         

In [27]:
text1

'```json\n["D001", "D004", "D002", "D003"]\n```'

In [29]:
import re, json
re.split(r'[,\\n]+', text1)
# [tok.strip().strip('"').strip("'") for tok in re.split(r'[,\\n]+', text1)]

['```jso', '\n["D001"', ' "D004"', ' "D002"', ' "D003"]\n```']

In [30]:
m = re.search(r'(\[.*\])', text1, re.S)
if m:
    ordered = json.loads(m.group(1))

ordered

# ordered = [tok.strip().strip('"').strip("'") for tok in re.split(r'[,\\n]+', text1) if tok.strip()]
# ordered

['D001', 'D004', 'D002', 'D003']

In [11]:
priority_map = {"high": 0, "medium": 1, "low": 2}
ordered_delivery = sorted(deliveries, key=lambda x: (priority_map.get(x.get("priority","medium"),1), x["id"]))
ordered_delivery

[i['id'] for i in ordered_delivery]


['D001', 'D002', 'D004', 'D003']

In [4]:
from agents import PlannerAgent, OptimizerAgent, MonitorAgent, DispatcherAgent

planner = PlannerAgent()
ordered_ids = planner.prioritize(deliveries, "")
ordered_ids

Initializing model...
Invoking LLM
Response Generated: ```json
["D001", "D004", "D002", "D003"]
```
creating ordered json
Completed ordering
LLM (openai/gpt-oss-120b) suggestion: ['D001', 'D004', 'D002', 'D003']


['D001', 'D004', 'D002', 'D003']

In [12]:
import random
def generate_random_orders(num_orders, bounds):
    """
    Generate a list of random delivery orders within given mapbox coordinates.

    Parameters:
        num_orders (int): Number of orders to generate.
        bounds (dict): Bounding box with min_lat, max_lat, min_lon, max_lon.

    Returns:
        list: List of randomly generated order dictionaries.
    """
    priorities = ["high", "medium", "low"]
    package_sizes = ["small", "medium", "large"]
    street_names = ["Park Street", "College Street", "Gariahat Road", "Salt Lake", "New Alipore", "Ballygunge", "Dum Dum", "Behala"]

    orders = []

    for i in range(1, num_orders + 1):
        lat = round(random.uniform(bounds["min_lat"], bounds["max_lat"]), 6)
        lon = round(random.uniform(bounds["min_lon"], bounds["max_lon"]), 6)
        address = f"{random.choice(street_names)}, Kolkata, India"
        order = {
            "id": f"D{str(i).zfill(3)}",
            "customer_name": random.choice(["Amit", "Rina", "Soumen", "Priya", "Kunal", "Sneha", "Rohit", "Mira"]),
            "address": address,
            "lat": lat,
            "lon": lon,
            "priority": random.choice(priorities),
            "package_size": random.choice(package_sizes),
            "fragile": random.choice([True, False])
        }
        orders.append(order)

    return orders


In [15]:
import random
import json

# Set your OpenAI API key

def generate_orders_with_llm(num_orders, bounds):
    """
    Generate realistic delivery orders using an LLM within given mapbox coordinates.
    """

    # Step 1: Generate random lat/lon pairs within bounds
    coords = [
        {
            "lat": round(random.uniform(bounds["min_lat"], bounds["max_lat"]), 6),
            "lon": round(random.uniform(bounds["min_lon"], bounds["max_lon"]), 6)
        }
        for _ in range(num_orders)
    ]

    # Step 2: Construct a system prompt for LLM
    system_prompt = (
        "You are a logistics data generator. "
        "Generate delivery order JSON objects for a courier company in Kolkata, India. "
        "Each order should include: id, customer_name, address, latitude, longitude, priority, package_size, fragile."
        "Use realistic Bengali or Indian names and real street/locality-style addresses in Kolkata."
        "Priorities should be 'high', 'medium', or 'low'. Package sizes: 'small', 'medium', 'large'."
        "Fragile is true or false. Return JSON only."
    )

    # Step 3: Combine lat/lon into the prompt
    user_prompt = f"Generate {num_orders} delivery orders for these coordinates:\n{json.dumps(coords, indent=2)}"

    # Step 4: Call the LLM
    response = llm1.invoke([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}],
    )

    # Step 5: Parse and return structured JSON
    try:
        orders = json.loads(response.content)
    except json.JSONDecodeError:
        print("⚠️ Could not parse JSON. Raw LLM output:")
        print(response.content)
        return []

    return orders


# Example usage:
kolkata_bounds = {
    "min_lat": 22.45,
    "max_lat": 22.66,
    "min_lon": 88.30,
    "max_lon": 88.42
}

orders = generate_orders_with_llm(5, kolkata_bounds)

for order in orders:
    print(order)


{'id': 1001, 'customer_name': 'Suman Ghosh', 'address': '123/4, Raja Rammohan Roy Road, Jadavpur, Kolkata, West Bengal 700032', 'latitude': 22.533526, 'longitude': 88.416739, 'priority': 'high', 'package_size': 'medium', 'fragile': False}
{'id': 1002, 'customer_name': 'Ananya Chakraborty', 'address': 'Flat B-12, Axis Mall, New Town, Kolkata, West Bengal 700156', 'latitude': 22.634773, 'longitude': 88.408392, 'priority': 'medium', 'package_size': 'large', 'fragile': True}
{'id': 1003, 'customer_name': 'Rohit Banerjee', 'address': '45/2, Tollygunge Club Road, Tollygunge, Kolkata, West Bengal 700033', 'latitude': 22.557888, 'longitude': 88.405312, 'priority': 'low', 'package_size': 'small', 'fragile': False}
{'id': 1004, 'customer_name': 'Mitali Dutta', 'address': '12/1, Behala Bazar Road, Behala, Kolkata, West Bengal 700061', 'latitude': 22.465342, 'longitude': 88.310994, 'priority': 'high', 'package_size': 'medium', 'fragile': True}
{'id': 1005, 'customer_name': 'Arindam Singh', 'addres

In [14]:
kolkata_bounds = {
    "min_lat": 22.45,
    "max_lat": 22.66,
    "min_lon": 88.30,
    "max_lon": 88.42
}

sample_orders = generate_random_orders(5, kolkata_bounds)
for order in sample_orders:
    print(order)


{'id': 'D001', 'customer_name': 'Mira', 'address': 'Dum Dum, Kolkata, India', 'lat': 22.637885, 'lon': 88.324873, 'priority': 'medium', 'package_size': 'medium', 'fragile': True}
{'id': 'D002', 'customer_name': 'Priya', 'address': 'Behala, Kolkata, India', 'lat': 22.471468, 'lon': 88.370643, 'priority': 'high', 'package_size': 'medium', 'fragile': True}
{'id': 'D003', 'customer_name': 'Rohit', 'address': 'Gariahat Road, Kolkata, India', 'lat': 22.618636, 'lon': 88.398738, 'priority': 'high', 'package_size': 'large', 'fragile': False}
{'id': 'D004', 'customer_name': 'Kunal', 'address': 'College Street, Kolkata, India', 'lat': 22.641677, 'lon': 88.369826, 'priority': 'low', 'package_size': 'large', 'fragile': True}
{'id': 'D005', 'customer_name': 'Rina', 'address': 'College Street, Kolkata, India', 'lat': 22.599709, 'lon': 88.328978, 'priority': 'high', 'package_size': 'medium', 'fragile': True}


In [16]:
my_list = ["apple", "banana", "cherry"]
str(my_list)

"['apple', 'banana', 'cherry']"

In [17]:
a = (22.4883, 88.379)
a[0]

22.4883

In [ ]:
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from sklearn_extra.cluster import KMedoids

def haversine_distance(coord1, coord2):
    """
    Compute Haversine distance (in km) between two (lat, lon) coordinates.
    """
    R = 6371  # Earth radius in kilometers
    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c


def cluster_delivery_points(coordinates, n_clusters):
    """
    Cluster delivery coordinates into n clusters using Haversine distance.

    Parameters:
        coordinates (list of tuples): [(lat, lon), (lat, lon), ...]
        n_clusters (int): Number of clusters to form.

    Returns:
        dict: {cluster_id: [list of coordinates in that cluster]}
    """
    # Convert to numpy array
    coords = np.array(coordinates)
    n_points = len(coords)

    # Compute pairwise Haversine distance matrix
    distance_matrix = np.zeros((n_points, n_points))
    for i in range(n_points):
        for j in range(i + 1, n_points):
            d = haversine_distance(coords[i], coords[j])
            distance_matrix[i][j] = d
            distance_matrix[j][i] = d

    # Cluster using K-Medoids with precomputed distances
    model = KMedoids(n_clusters=n_clusters, metric='precomputed', random_state = 42)
    model.fit(distance_matrix)

    # Group coordinates by cluster
    clusters = {}
    for i, label in enumerate(model.labels_):
        clusters.setdefault(label, []).append(tuple(coords[i]))

    return clusters


In [1]:
import numpy as np
import hdbscan
import folium

def cluster_delivery_points_hdbscan(coordinates, min_cluster_size=3, eps_km=5):
    """
    Cluster delivery coordinates using HDBSCAN with Haversine distance.

    Parameters:
        coordinates (list of tuples): [(lat, lon), (lat, lon), ...]
        min_cluster_size (int): Minimum cluster size.
        eps_km (float): Approximate neighborhood size in kilometers (used to set min_samples).

    Returns:
        dict: {cluster_id: [coordinates]} and list of labels for each point.
    """
    # Convert coordinates to radians for Haversine metric
    coords_radians = np.radians(np.array(coordinates))

    # HDBSCAN clustering
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=1,
        metric='haversine',
        cluster_selection_epsilon=eps_km / 6371.0  # convert km to radians
    )
    cluster_labels = clusterer.fit_predict(coords_radians)

    # Group by cluster ID
    clusters = {}
    for idx, label in enumerate(cluster_labels):
        if label == -1:
            # -1 = noise points
            clusters.setdefault("noise", []).append(coordinates[idx])
        else:
            clusters.setdefault(label, []).append(coordinates[idx])

    return clusters, cluster_labels


In [2]:
import random

def visualize_clusters_on_map(coordinates, cluster_labels):
    """
    Visualize clustered delivery points on a Folium map with color-coded markers.
    """
    # Center map around mean coordinates
    avg_lat = np.mean([lat for lat, lon in coordinates])
    avg_lon = np.mean([lon for lat, lon in coordinates])
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    # Assign random colors to clusters
    unique_labels = set(cluster_labels)
    colors = {
        label: f"#{random.randint(0, 0xFFFFFF):06x}"
        for label in unique_labels if label != -1
    }
    colors[-1] = "black"  # noise points

    # Add points to map
    for (lat, lon), label in zip(coordinates, cluster_labels):
        color = colors.get(label, "gray")
        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            color=color,
            fill=True,
            fill_opacity=0.8,
            tooltip=f"Cluster {label}" if label != -1 else "Noise"
        ).add_to(m)

    return m


In [5]:
delivery_points = [
(22.619085,88.403029),
(22.615411,88.343062),
(22.640267,88.401875),
(22.595167,88.359004),
(22.633627,88.416699),
(22.599717,88.378695),
]

In [35]:
clusters, labels = cluster_delivery_points_hdbscan(delivery_points, min_cluster_size=3)
labels[0]


c:\Projects\route_optimizer\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Projects\route_optimizer\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


np.int64(0)

In [ ]:

# Visualize
m = visualize_clusters_on_map(delivery_points, labels)
m.save("clustered_deliveries.html")


c:\Projects\route_optimizer\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Projects\route_optimizer\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [12]:
import config

coordinates = [(order["lat"], order["lon"]) for order in config.deliveries]

In [13]:
avg_lat = np.mean([lat for lat, lon in coordinates])
avg_lat

np.float64(22.617212333333338)

In [14]:
np.radians(np.array(coordinates))

array([[0.39477751, 1.54292392],
       [0.39471338, 1.5418773 ],
       [0.3951472 , 1.54290378],
       [0.39436006, 1.54215554],
       [0.39503131, 1.54316251],
       [0.39443947, 1.54249922]])

In [26]:
unique_labels = set(labels)
colors = {
    label: f"#{random.randint(0, 0xFFFFFF):06x}"
    for label in unique_labels if label != -1
}
colors[-1] = "black"  # noise points

for (lat, lon), label in zip(coordinates, labels):
    color = colors.get(label, "gray")
    print(color)

#c301a1
#51b523
#c301a1
#51b523
#c301a1
#51b523


In [36]:
colors

{np.int64(0): '#c301a1', np.int64(1): '#51b523', -1: 'black'}

In [28]:
color = colors.get(1, "gray")
color

'#51b523'

In [44]:
from agents import ClusteringAgent
import config

clusterer = ClusteringAgent()

deliveries = config.load_json(config.DELIVERIES_FILE)


In [45]:
cluster, label = clusterer.cluster_delivery_points_hdbscan(deliveries)

c:\Projects\route_optimizer\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Projects\route_optimizer\.venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [46]:
cluster

{'Cluster_0': [{'id': 'ORD1001',
   'customer_name': 'Arindam Chatterjee',
   'address': '12/3, Beniapukur Road, Beniapukur, Kolkata, West Bengal 700015',
   'lat': 22.551332,
   'lon': 88.441328,
   'priority': 'high',
   'package_size': 'medium',
   'fragile': False},
  {'id': 'ORD1003',
   'customer_name': 'Sourav Banerjee',
   'address': '8A, Ballygunge Circular Road, Ballygunge, Kolkata, West Bengal 700019',
   'lat': 22.513659,
   'lon': 88.426829,
   'priority': 'low',
   'package_size': 'small',
   'fragile': False},
  {'id': 'ORD1005',
   'customer_name': 'Kunal Mukherjee',
   'address': '21/7, New Town, Action Area I, Kolkata, West Bengal 700156',
   'lat': 22.555112,
   'lon': 88.429076,
   'priority': 'medium',
   'package_size': 'medium',
   'fragile': False},
  {'id': 'ORD1010',
   'customer_name': 'Namita Chakraborty',
   'address': '27/6, College Street, College Row, Kolkata, West Bengal 700073',
   'lat': 22.544052,
   'lon': 88.44476,
   'priority': 'low',
   'package

In [50]:
aa= [1, 4, 12, 18, 20, 5, 3]

' > '.join(str(num) for num in aa)

'1 > 4 > 12 > 18 > 20 > 5 > 3'